# Imports

In [1]:
import os
from ansys.edb.database import Database
from ansys.edb.session import launch_session
import ansys.edb as edb
from ansys.edb.definition.djordjecvic_sarkar_model import DjordjecvicSarkarModel
import json

# Configs

In [2]:
ROOT = os.getcwd()
EXE_ROOT = os.environ['ANSYSEM_ROOT232']
EDB_FILE = r'D:\2023\PYEDB\Galileo.aedb'

# Launch session

In [3]:
session = launch_session(EXE_ROOT, 50051)

# Open DB

In [4]:
db = Database.open(EDB_FILE,False)
cell = db.circuit_cells[0]
layout = cell.layout
layerStats = []
lc = layout.layer_collection
layers = lc.get_layers()
layer_names = [i.name for i in layers]

# Extracting layer by layer information

In [5]:
layer_info = []
for i in layers:
    if i.is_stackup_layer:
        name = i.name
        layer_type = i.type.name
        material = i.get_material().value
        thickness = i.thickness.value
        lower_elevation = i.lower_elevation.value
        full_material = i.get_fill_material().value
        
        if 'DIELECTRIC_LAYER' in str(layer_type):
            matmodel = edb.definition.material_def.MaterialDef.find_by_name(db,i.get_material().value)
            ds = DjordjecvicSarkarModel.create()
            matmodel.dielectric_material_model=ds            
        
db.save_as(r'D:\2023\PYEDB\Galileo_modified.aedb')

# Write layer information to json file

In [6]:
stackup_file = os.path.join(os.getcwd(),'stackup.json')
with open(stackup_file, 'w') as fp:
    json.dump(layer_info, fp,indent=' ')

# End the session

In [7]:
session.disconnect()